In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings

warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

In [ ]:
data_df = pd.read_pickle("../input/candlestick-eda/data_df.pkl")
print(data_df.shape)
data_df[:3]

In [ ]:
def define_model():
    keras.backend.clear_session()

    input_img = keras.Input(shape=(40, 40, 1))
#     encoded_img = keras.Input(shape=(5, 5, 4))
    
    x = layers.Conv2D(8, (2, 2), activation=None, padding='same')(input_img)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x)

    autoencoder = keras.Model(input_img, decoded)
    
    encoder = keras.Model(input_img, encoded)
    
    # decoder_layers = autoencoder.layers[-10:]
    # decoder = keras.Model(encoded_img, decoder_layers(decoded))
    decoder = None
    
    return encoder, decoder, autoencoder

encoder, decoder, autoencoder = define_model()
autoencoder.summary()

In [ ]:
X = data_df.sort_values("imgID")["imgData"].apply(lambda x: x.reshape(1,40,40,1))
X = np.concatenate(X.values)
X.shape

In [ ]:
np.random.seed(1291)

encoder, decoder, autoencoder = define_model()
adam = keras.optimizers.Adam(learning_rate=0.001)
autoencoder.compile(optimizer=adam, loss='binary_crossentropy')

es = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.00001,
    patience=15,
    verbose=1,
    mode='auto',
    restore_best_weights=True)
ckp = keras.callbacks.ModelCheckpoint(
    filepath="best_model.ckp",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

autoencoder.fit(X, X, 
                batch_size=128,
                epochs=1500,
                verbose=1,
                validation_split=0.2,
                callbacks=[es, ckp] 
               )

In [ ]:
plt.plot(autoencoder.history.history["loss"], ".:")
plt.plot(autoencoder.history.history["val_loss"], ".:")

In [ ]:
reconstucts = autoencoder.predict(X[:15])

In [ ]:
for i in range(15):
    plt.imshow(np.concatenate([X[i],reconstucts[i]],1))
    plt.show()

In [ ]:
encoder.predict(X[:5]).shape